In [1]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from pycuda.compiler import SourceModule

This does not work, it needs to be worked out and tested against Trimesh be

In [4]:
# Define CUDA kernel to calculate closest point on a triangle
kernel_code = """
__device__ float3 cross_product(float3 a, float3 b) {
    return make_float3(a.y * b.z - a.z * b.y,
                       a.z * b.x - a.x * b.z,
                       a.x * b.y - a.y * b.x);
}

__device__ float dot_product(float3 a, float3 b) {
    return a.x * b.x + a.y * b.y + a.z * b.z;
}

__device__ float3 subtract(float3 a, float3 b) {
    return make_float3(a.x - b.x, a.y - b.y, a.z - b.z);
}

__device__ float3 add(float3 a, float3 b) {
    return make_float3(a.x + b.x, a.y + b.y, a.z + b.z);
}

__device__ float3 scale(float3 v, float s) {
    return make_float3(v.x * s, v.y * s, v.z * s);
}

__device__ bool point_in_triangle(float3 p, float3 v0, float3 v1, float3 v2) {
    // Barycentric coordinates method to check if the point is inside the triangle
    float3 v0v1 = subtract(v1, v0);
    float3 v0v2 = subtract(v2, v0);
    float3 v0p = subtract(p, v0);
    
    float d00 = dot_product(v0v1, v0v1);
    float d01 = dot_product(v0v1, v0v2);
    float d11 = dot_product(v0v2, v0v2);
    float d20 = dot_product(v0p, v0v1);
    float d21 = dot_product(v0p, v0v2);
    
    float denom = d00 * d11 - d01 * d01;
    float v = (d11 * d20 - d01 * d21) / denom;
    float w = (d00 * d21 - d01 * d20) / denom;
    float u = 1.0f - v - w;
    
    return (u >= 0.0f) && (v >= 0.0f) && (w >= 0.0f);
}

__device__ float3 closest_point_on_triangle(float3 p, float3 v0, float3 v1, float3 v2) {
    // Compute the closest point using perpendicular projection
    float3 v0v1 = subtract(v1, v0);
    float3 v0v2 = subtract(v2, v0);
    float3 v0p = subtract(p, v0);
    
    float d00 = dot_product(v0v1, v0v1);
    float d01 = dot_product(v0v1, v0v2);
    float d11 = dot_product(v0v2, v0v2);
    float d20 = dot_product(v0p, v0v1);
    float d21 = dot_product(v0p, v0v2);
    
    float denom = d00 * d11 - d01 * d01;
    float v = (d11 * d20 - d01 * d21) / denom;
    float w = (d00 * d21 - d01 * d20) / denom;
    float u = 1.0f - v - w;
    
    float3 closest_point = add(v0, add(scale(v0v1, u), scale(v0v2, v)));
    return closest_point;
}

__global__ void find_closest_point_and_normal(float3 *vertices, int3 *indices, float3 query_point, 
                                               float3 *closest_point, float3 *normals, int num_triangles) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= num_triangles) return;
    
    int3 triangle = indices[idx];
    
    float3 v0 = vertices[triangle.x];
    float3 v1 = vertices[triangle.y];
    float3 v2 = vertices[triangle.z];
    
    // Find closest point on triangle
    float3 point_on_triangle = closest_point_on_triangle(query_point, v0, v1, v2);
    
    // Check if this point is closer than the current closest point
    float3 closest = *closest_point;
    float dist_new = dot_product(subtract(point_on_triangle, query_point), subtract(point_on_triangle, query_point));
    float dist_old = dot_product(subtract(closest, query_point), subtract(closest, query_point));
    
    if (dist_new < dist_old || dist_old == 0.0f) {
        *closest_point = point_on_triangle;
        
        // Compute normal of the triangle
        float3 edge1 = subtract(v1, v0);
        float3 edge2 = subtract(v2, v0);
        float3 normal = cross_product(edge1, edge2);
        float norm_length = sqrt(dot_product(normal, normal));
        normal = scale(normal, 1.0f / norm_length);  // Normalize the normal
        
        *normals = normal;
    }
}
"""

# Compile the CUDA kernel
mod = SourceModule(kernel_code)

In [15]:
# Initialize GPU memory
def closest_point_on_mesh(vertices, triangles, query_point):
    num_triangles = len(triangles)
    
    # Allocate memory on GPU
    vertex_gpu = cuda.mem_alloc(vertices.nbytes)
    triangle_gpu = cuda.mem_alloc(triangles.nbytes)
    query_gpu = cuda.mem_alloc(query_point.nbytes)
    closest_point_gpu = cuda.mem_alloc(query_point.nbytes)
    normals_gpu = cuda.mem_alloc(query_point.nbytes)

    # Transfer data to GPU
    cuda.memcpy_htod(vertex_gpu, vertices)
    cuda.memcpy_htod(triangle_gpu, triangles)
    cuda.memcpy_htod(query_gpu, query_point)
    
    # Initialize closest point to a large value
    initial_point = np.array([float('inf'), float('inf'), float('inf')], dtype=np.float32)
    cuda.memcpy_htod(closest_point_gpu, initial_point)
    
    # Launch the kernel
    block_size = 256
    grid_size = (num_triangles + block_size - 1) // block_size
    func = mod.get_function("find_closest_point_and_normal")
    func(vertex_gpu, triangle_gpu, query_gpu, closest_point_gpu, normals_gpu, np.int32(num_triangles), 
         block=(block_size, 1, 1), grid=(grid_size, 1))

    # Retrieve results from GPU
    closest_point_result = np.empty_like(query_point)
    cuda.memcpy_dtoh(closest_point_result, closest_point_gpu)
    normals_result = np.empty_like(query_point)
    cuda.memcpy_dtoh(normals_result, normals_gpu)

    return closest_point_result, normals_result

# Example usage
vertices = np.array([
    [0.0, 0.0, 0.0],  # Vertex 0
    [1.0, 0.0, 0.0],  # Vertex 1
    [0.0, 1.0, 0.0],  # Vertex 2
    [0.0, 0.0, 1.0],  # Vertex 3
], dtype=np.float32)

triangles = np.array([
    [0, 1, 2],  # Triangle 1
    [0, 1, 3],  # Triangle 2
], dtype=np.int32)

query_point = np.array([0.1, 0.1, 0.1], dtype=np.float32)

closest_point, normal = closest_point_on_mesh(vertices, triangles, query_point)
print("Closest Point:", closest_point)
print("Normal:", normal)

Closest Point: [inf inf inf]
Normal: [0. 0. 0.]


In [13]:
from pygments import highlight
from pygments.lexers import CppLexer
from pygments.formatters import HtmlFormatter
from IPython.core.display import HTML

In [14]:
formatter = HtmlFormatter(style="colorful", full=True, noclasses=True)
highlighted_code = highlight(kernel_code, CppLexer(), formatter)

HTML(highlighted_code)